In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
base = automap_base()
# reflect the tables
base.prepare(autoload_with=engine)

In [6]:
# View all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurements = base.classes.measurement
stations = base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set
most_recent_date = session.query(func.max(measurements.date)).scalar()
print(most_recent_date)


2017-08-23


In [10]:
# Design a query to retrieve the last 12 montsh of precipitation data and plot the results.
# Starting from the most recent data point in the database.


# Change most recent date format 
most_recent_date = pd.to_datetime(most_recent_date)

# Calculate the date one year from the last date in data set.
twelve_months_ago = most_recent_date - pd.DateOffset(months=12)

# Remove minutes and seconds from the date
twelve_months_ago = twelve_months_ago.date()

# Print the date
print(twelve_months_ago)

2016-08-23


In [11]:
# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(measurements.date, measurements.prcp).filter(measurements.date >= twelve_months_ago).all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
precip_df = pd.DataFrame(precipitation_data, columns=['Date', 'Precipitation'])

# Sort the dataframe by date
precip_df = precip_df.sort_values(by='Date', ascending=True)

# Use Pandas Plotting with Matplotlib to plot the data
precip_df.plot(x='Date', y='Precipitation', kind='line', figsize=(12, 6))

plt.title("Precipitation over 12 Months")
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")
plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig('precip_line_chart.png')

plt.show()

In [12]:
# Use Pandas to calculate the summary statistics for the precipitation data
precip_summary_stats = precip_df['Precipitation'].describe()
precip_summary_stats

count    2021.000000
mean        0.177279
std         0.461190
min         0.000000
25%         0.000000
50%         0.020000
75%         0.130000
max         6.700000
Name: Precipitation, dtype: float64

# Exploratory Station Analysis

In [13]:
# Design a query to calculate the total number of stations in the dataset
stations_count = session.query(stations.station).count()
stations_count

9

In [15]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
most_active_stations = session.query(measurements.station, func.count(measurements.station)).group_by(measurements.station).order_by(func.count(measurements.station).desc()).all()
most_active_stations

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [ ]:
# Most active station id
most_active_station_id = most_active_stations [0][0]
most_active_station_id

'USC00519281'

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

temp_low_high_avg = session.query(
    func.min(measurements.tobs),
    func.max(measurements.tobs),
    func.avg(measurements.tobs)
).filter(measurements.station == most_active_station_id).all()
temp_low_high_avg

[(54.0, 85.0, 71.66378066378067)]

In [21]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_data = session.query(measurements.tobs)\
.filter(measurements.station == most_active_station_id)\
.filter(measurements.date >= twelve_months_ago) \
.all()

temperatures = [temp[0] for temp in temp_data]

plt.figure(figsize=(6,5))
plt.hist(temperatures, bins=12)
plt.title(f'Temps for Most Active Station')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()

plt.savefig('temps_most_active_station.png')

plt.show()

# Close Session

In [22]:
# Close Session
session.close()